In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Первый взгляд на данные

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.info()
print('-' * 100)
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
# пропуски есть
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
# дубликатов нет
df_train.duplicated().sum()

In [ ]:
df_test.duplicated().sum()

# 2. EDA и первичная обработка данных

In [ ]:
# посмотрим на целевую переменную
# классы целевой переменной относительно сбалансированы
df_train['Survived'].value_counts()

In [ ]:
# признаки PassengerId и Ticket вряд ли несут полезную информацию, удалим их
df_train.drop(columns=['PassengerId', 'Ticket'], inplace=True)
df_test.drop(columns=['PassengerId', 'Ticket'], inplace=True)

In [ ]:
# оценим корреляцию между числовыми признаками
sns.heatmap(df_train.corr(numeric_only=True), annot=True);
# существенной корреляции целевой переменной с другими не наблюдается
# сильной корреляции между различными признаками тоже нет

In [ ]:
# оценим связь числовых переменных
sns.pairplot(data=df_train, hue='Survived');

In [ ]:
sns.countplot(x='Pclass', data=df_train, hue='Survived');
# выживаемость зависит от класса билета

In [ ]:
# из признака Name теоретически можно извлечь титул или еще что-то, но не будем этим заниматься и удалим его
df_train.drop(columns=['Name'], inplace=True)
df_test.drop(columns=['Name'], inplace=True)

In [ ]:
sns.countplot(x='Sex', data=df_train, hue='Survived');
# выживаемость зависит от пола

In [ ]:
sns.countplot(x='Sex', data=df_train, hue='Pclass');
# мужчины в основном были в 3 классе

In [ ]:
# посмотрим на распределение возрастов
sns.histplot(x='Age', data=df_train, kde=True);
# стариков больше, чем детей

In [ ]:
# преобразуем Age в диапазон возрастов
# labels=False возвращает только целочисленные индикаторы
# разбиение для bins=5 показывает, что в самой молодой группе выживших больше, чем погибших, оставим его
df_train['Age_group'] = pd.cut(df_train['Age'], bins=5, labels=False)
sns.countplot(x='Age_group', data=df_train, hue='Survived');

In [ ]:
# посмотрим, на какие возрастные группы делается разбиение при bins=5
pd.cut(df_train['Age'], bins=5).value_counts()

In [ ]:
sns.boxplot(x='Pclass', y='Age', data=df_train);
# чем выше класс, тем выше возраст пассажиров (медианный)

In [ ]:
# сделаем один признак Relatives как сумму SibSp и Parch, старые признаки удалим
df_train['Relatives'] = df_train['SibSp'] + df_train['Parch']
df_test['Relatives'] = df_test['SibSp'] + df_test['Parch']

df_train.drop(columns=['SibSp', 'Parch'], inplace=True)
df_test.drop(columns=['SibSp', 'Parch'], inplace=True)

In [ ]:
sns.boxplot(x='Relatives', y='Age', data=df_train);
# чем больше родственников, тем ниже возраст пассажиров (медианный)

In [ ]:
sns.countplot(x='Relatives', data=df_train, hue='Survived');

In [ ]:
sns.histplot(x='Fare', data=df_train, kde=True);

In [ ]:
sns.boxplot(x='Pclass', y='Fare', data=df_train);
# большой разброс цен на билеты 1 класса

In [ ]:
sns.regplot(x='Age', y='Fare', data=df_train);
# корреляция между ценой билета и возрастом довольно слабая

In [ ]:
# признак Cabin содержит названия палуб (первые буквы), но в нем очень много пропусков
df_train['Cabin'].unique()

In [ ]:
# создадим новый признак с названиями палуб из столбца Cabin
df_train['Deck'] = df_train['Cabin'].str[0]
df_train.head()

In [ ]:
sns.countplot(x='Deck', data=df_train, hue='Survived');
# вряд ли получится извлечь много пользы от этого, к тому же много пропусков, поэтому удалим и Cabin, и Deck

In [ ]:
df_train.drop(columns=['Cabin', 'Deck'], inplace=True)
df_test.drop(columns=['Cabin'], inplace=True)

In [ ]:
sns.countplot(x='Embarked', data=df_train, hue='Survived');
# пассажиры из порта C выживали чаще, чем погибали, возможно, это связано с их расположением на корабле

In [ ]:
# удалим ранее созданный признак Age_group из train
df_train.drop(columns='Age_group', inplace=True)

**Выводы**

Очевидно, что признаки 'Pclass', 'Sex', 'Age' оказывают существенное влияние на целевую переменную

# 3. Обработка данных

In [ ]:
# разделим данные на признаки и таргет
X_train = df_train.drop(columns='Survived')
y_train = df_train['Survived']
X_test = df_test

**3.1 Обработка пропусков**

В train - пропуски в столбцах Age и Embarked, в test - в Age и Fare

In [ ]:
# для заполнения пропусков Age как в числовом признаке используем KNNImputer
# т.к. используется метрический метод, данные нужно сначала нормализовать
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# создадим датафреймы только с интересующими нас признаками
X_train_knn = X_train.copy()[['Pclass','Age', 'Fare']] 
X_test_knn = X_test.copy()[['Pclass','Age', 'Fare']]

# нормализуем данные - обучаем StandardScaler на train, применяем его же на test
scaler_knn = StandardScaler()
X_train_knn = pd.DataFrame(scaler_knn.fit_transform(X_train_knn), columns=X_train_knn.columns)
X_test_knn = pd.DataFrame(scaler_knn.transform(X_test_knn), columns=X_test_knn.columns)

X_train_knn.head()

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform')

# обучаем KNNImputer на train, далее применяем его к test (чтобы избежать утечки данных)
X_train_knn = pd.DataFrame(knn_imputer.fit_transform(X_train_knn), columns=X_train_knn.columns)
X_test_knn = pd.DataFrame(knn_imputer.transform(X_test_knn), columns=X_test_knn.columns)

In [ ]:
# вернем исходный масштаб данных
X_train_knn = pd.DataFrame(scaler_knn.inverse_transform(X_train_knn), columns=X_train_knn.columns)
X_test_knn = pd.DataFrame(scaler_knn.inverse_transform(X_test_knn), columns=X_test_knn.columns)

X_train_knn.head()

In [ ]:
# подставим заполненные значения в исходные датафреймы
mask_train = X_train['Age'].isnull()
mask_test = X_test['Age'].isnull()
X_train.loc[mask_train, 'Age'] = X_train_knn.loc[mask_train]
X_test.loc[mask_test, 'Age'] = X_test_knn.loc[mask_test]

In [ ]:
# также для заполнения пропусков в Age можно применить следующий подход
"""
# для заполнения Age сгруппируем людей по Pclass и возьмем медиану в каждой группе
median_ages_by_class = X_train.groupby('Pclass')['Age'].median()

def Age_filler(x):
    if pd.isna(x['Age']):
        return median_ages_by_class[x['Pclass']]
    return x['Age']


# мы применяем функцию по столбцам - т.е. в Age_filler передается серия, состоящая из одной строки датафрейма
X_train['Age'] = X_train.apply(Age_filler, axis=1)
X_test['Age'] = X_test.apply(Age_filler, axis=1)
"""

In [ ]:
# пропуски в Embarked и Fare заполним модой и медианой соответственно
# мы не нарушаем принципа разделения данных, поскольку меняем test так же, как train
X_train['Embarked'].fillna(X_train['Embarked'].mode()[0], inplace=True)
X_test['Fare'].fillna(X_test['Fare'].median(), inplace=True)

In [ ]:
# пропусков теперь нет
X_train.isnull().sum().sum(), X_test.isnull().sum().sum()

**3.2 Работа с выбросами**

In [ ]:
# числовые признаки, для которых есть смысл смотреть выбросы - это Age и Fare
X_train.head()

In [ ]:
# посмотрим, какой процент от всех данных составляют выбросы
outliers_features = ['Age', 'Fare']
def outlier_count(X):
    Q25, Q75 = X.quantile(0.25, axis=0), X.quantile(0.75, axis=0)
    IQR = Q75 - Q25
    lower_bound, upper_bound = Q25 - 1.5 * IQR, Q75 + 1.5 * IQR
    quantity = ((X >= upper_bound) | (X <= lower_bound)).sum(axis=0) / X.shape[0] * 100 # доля выбросов для каждого признака в процентах
    return quantity.sort_values(ascending=False)


outlier_count(X_train[outliers_features])
# достаточно большое число выбросов в признаке Fare
# пока оставим, как есть

**3.3 Кодирование категориальных переменных**

Общие соображения:
1. Удобный инструмент category_encoders как будто не может удалить один лишний столбец, наличие которого может приводить к мультиколлинеарности (либо я не нашел способ это сделать). При этом если в test появляются новые данные, то обученный на train category_encoders может присваивать им нулевые значения, как и должно быть (задается через handle_unknown='value' - значение по умолчанию).
2. Метод pandas.get_dummies не «запоминает» категории при обучении - т.е. если значения в train и test будут отличаться, появится ошибка.

Исходя из этого, применим OneHotEncoder, в котором можно указать параметр handle_unknown='ignore'. В этом случае при обнаружении неизвестной категории во время преобразования столбец будет содержать все нули. Также через drop='first' удаляем первый (избыточный) признак

In [ ]:
# необходимо закодировать признаки Pclass, Sex и Embarked
# спорный вопрос, стоит ли кодировать Pclass - с одной стороны, у нас нет уверенности, что первый класс в 2 раза лучше второго и т.д.
# с другой - при кодировании мы теряем информацию о разнице между классами
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder(drop='first', sparse_output = False, handle_unknown='ignore') # удаляем первый (избыточный) признак через drop='first'
encoded_X_train = pd.DataFrame(onehotencoder.fit_transform(X_train[['Pclass', 'Sex', 'Embarked']])) # нужно передавать двумерный массив
encoded_X_train.columns = onehotencoder.get_feature_names_out() # добавляем названия столбцов через метод get_feature_names_out

X_train = X_train.join(encoded_X_train) # присоединим новые признаки к исходному датафрейму
X_train.drop(columns=['Pclass', 'Sex', 'Embarked'], inplace=True) # удалим исходные признаки

X_train

In [ ]:
# то же самое для test
encoded_X_test = pd.DataFrame(onehotencoder.transform(df_test[['Pclass', 'Sex', 'Embarked']]))
encoded_X_test.columns = onehotencoder.get_feature_names_out()

X_test = df_test.join(encoded_X_test)
X_test.drop(columns=['Pclass', 'Sex', 'Embarked'], inplace=True)

X_test

In [ ]:
"""
# вариант через библиотеку category_encoders
import category_encoders as ce

# в параметр cols передадим столбцы, которые нужно преобразовать
# в метод .fit_transform() передадим весь датафрейм целиком
# старые столбцы автоматически удаляются
ohe_encoder = ce.OneHotEncoder(cols=['Pclass', 'Sex', 'Embarked'])
X_train = ohe_encoder.fit_transform(X_train)
"""

**3.4 Масштабирование и преобразование числовых признаков**

Необходимо нормализовать числовые признаки, используем для этого StandardScaler. Нет смысла масштабировать категориальные (булевы) признаки - это может необоснованно увеличить или уменьшить их интервал значений.
Соответственно, работаем только с числовыми признаками 'Age', 'Fare', 'Relatives'

In [ ]:
# сначала посмотрим, насколько числовые признаки близки к нормальному распределению
from scipy.stats import kstest, shapiro, skew 

num_features = ['Age', 'Fare', 'Relatives']
def norm_distr_check(X):
    df_norm_distr = pd.DataFrame(columns=['Критерий Колмогорова-Смирнова (p-value)', 
                                          'Критерий Шапиро-Уилка (p-value)', 
                                          'Коэффициент асимметрии']
                                            ) # пустой фрейм под результаты
    for col in X.columns:
        df_norm_distr.loc[col, 'Критерий Колмогорова-Смирнова (p-value)'] = kstest(X[col], 'norm').pvalue
        df_norm_distr.loc[col, 'Критерий Шапиро-Уилка (p-value)'] = shapiro(X[col]).pvalue
        df_norm_distr.loc[col, 'Коэффициент асимметрии'] = skew(X[col])
        
    return df_norm_distr


norm_distr_check(X_train[num_features])
# можно сделать вывод, что числовые признаки имеют отличное от нормального распределение
# в данном случае менять форму распределения не будем

In [ ]:
# оценим распределения числовых признаков визуально
def feature_vis(X):
    n_cols = X.shape[-1]
    fig, ax = plt.subplots(1, n_cols, figsize=(10, 6))
    ax = ax.flatten() # делаем массив осей одномерным

    for i, col in enumerate(X.columns):
        sns.histplot(X[col], ax=ax[i], kde=True, color="skyblue")
        ax[i].set_title(col)

    plt.tight_layout()

feature_vis(X_train[num_features])

In [ ]:
# попробем оставить признак Pclass в первоначальном виде (без кодирования)
# эту ячейку можно закомментировать, тогда признак будет кодироваться
X_train['Pclass'] = pd.read_csv('/kaggle/input/titanic/train.csv')['Pclass']
X_train = X_train.drop(columns=['Pclass_2', 'Pclass_3'])
X_test['Pclass'] = pd.read_csv('/kaggle/input/titanic/test.csv')['Pclass']
X_test = X_test.drop(columns=['Pclass_2', 'Pclass_3'])

X_test

In [ ]:
from sklearn.preprocessing import StandardScaler

scaled_features = ['Age', 'Fare', 'Relatives', 'Pclass'] # Pclass здесь указывается для варианта, когда он не кодируется
scaler = StandardScaler()
X_train_scaled = X_train.drop(columns=scaled_features).join(pd.DataFrame(scaler.fit_transform(X_train[scaled_features]), columns=scaled_features))
X_test_scaled = X_test.drop(columns=scaled_features).join(pd.DataFrame(scaler.transform(X_test[scaled_features]), columns=scaled_features))

In [ ]:
X_train_scaled

In [ ]:
X_test_scaled

In [ ]:
# финальная проверка на корректность данных
X_train_scaled.info()
print('-' * 100)
X_test_scaled.info()

In [ ]:
X_train_scaled.isnull().sum().sum(), X_test_scaled.isnull().sum().sum()

**3.5 Конструирование признаков**

В данном случае дополнительные признаки не создаются, ранее на этапе EDA был создан признак Relatives.

Попробуем применить PCA, чтобы оценить вклад признаков

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train_scaled)

# процент дисперсии, объясняемый каждым из выбранных компонентов (собств. числа ковариационной матрицы)
pca.explained_variance_ratio_

# 4. Моделирование и оценка результата

**4.1 KNN**

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier

scores = pd.DataFrame() # датафрейм под все модели

def objective(trial):
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    n_neighbors = trial.suggest_int('n_neighbors', 2, 15, step=1)
    knn_optuna = KNeighborsClassifier(weights=weights, n_neighbors=n_neighbors)
    score = cross_val_score(knn_optuna, X_train_scaled, y_train, cv=5).mean() # максимизируем одну метрику (в данном случае - accuracy)
    return score


def model_score(model):
    # здесь оценим все метрики классификации
    # переменная score возвращает словарь с результатами метрик
    score = cross_validate(model, X_train_scaled, y_train, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])
    return {k: v.mean() for k, v in score.items()}


study = optuna.create_study(direction='maximize', study_name='KNN')
study.optimize(objective, n_trials=100)

knn_optuna_score = study.best_value
knn_optuna_params = study.best_params

scores.loc['KNN', 'Score'] = knn_optuna_score
scores.loc['KNN', 'Model_params'] = str([f'{k}: {v}' for k, v in knn_optuna_params.items()])

In [ ]:
knn_optuna_params

In [ ]:
knn = KNeighborsClassifier(weights='uniform', n_neighbors=5)
model_score(knn)

In [ ]:
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)
# в качестве индекса используем PassengerId из test
idx = pd.read_csv('/kaggle/input/titanic/test.csv')['PassengerId']
submission_knn = pd.DataFrame({'PassengerId': idx, 'Survived': y_pred_knn})
submission_knn.to_csv("submission_knn.csv", index = False)
print('✅ submission_knn.csv created successfully!')

# результат на test - 0.72009 с кодированием Pclass
# результат на test - 0.72248 без кодирования Pclass

**4.2 SVM**

In [ ]:
from sklearn.svm import SVC

def objective(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    C = trial.suggest_float('C', 1e-4, 1e4, log=True)
    # фиксируем random_state для моделей, в которых он есть
    svm_optuna = SVC(kernel=kernel, C=C, random_state=42)
    score = cross_val_score(svm_optuna, X_train_scaled, y_train, cv=5).mean()
    return score


study = optuna.create_study(direction='maximize', study_name='SVM')
study.optimize(objective, n_trials=100)

svm_optuna_score = study.best_value
svm_optuna_params = study.best_params

scores.loc['SVM', 'Score'] = svm_optuna_score
scores.loc['SVM', 'Model_params'] = str([f'{k}: {v}' for k, v in svm_optuna_params.items()])

In [ ]:
svm_optuna_params

In [ ]:
svm = SVC(kernel='rbf', C=3.406575550158352, random_state=42)
model_score(svm)

In [ ]:
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/titanic/test.csv')['PassengerId']
submission_svm = pd.DataFrame({'PassengerId': idx, 'Survived': y_pred_svm})
submission_svm.to_csv("submission_svm.csv", index = False)
print('✅ submission_svm.csv created successfully!')

# результат на test - 0.77751 с кодированием Pclass
# результат на test - 0.76555 без кодирования Pclass

**4.3 Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 30, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 30, step=1)
    rf_optuna = RandomForestClassifier(criterion=criterion, n_estimators=n_estimators, 
                                       min_samples_split=min_samples_split, 
                                       min_samples_leaf=min_samples_leaf, random_state=42)
    score = cross_val_score(rf_optuna, X_train_scaled, y_train, cv=5).mean()
    return score
    

study = optuna.create_study(direction='maximize', study_name='Random Forest')
study.optimize(objective, n_trials=100)

rf_optuna_score = study.best_value
rf_optuna_params = study.best_params

scores.loc['Random Forest', 'Score'] = rf_optuna_score
scores.loc['Random Forest', 'Model_params'] = str([f'{k}: {v}' for k, v in rf_optuna_params.items()])

In [ ]:
rf_optuna_params

In [ ]:
rf = RandomForestClassifier(criterion='gini', n_estimators=200,
                                       min_samples_split=4, 
                                       min_samples_leaf=3, random_state=42)
model_score(rf)

In [ ]:
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/titanic/test.csv')['PassengerId']
submission_rf = pd.DataFrame({'PassengerId': idx, 'Survived': y_pred_rf})
submission_rf.to_csv("submission_rf.csv", index = False)
print('✅ submission_rf.csv created successfully!')

# результат на test - 0.77033 с кодированием Pclass
# результат на test - 0.77511 без кодирования Pclass

**4.4 AdaBoost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=10)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1, log=True)
    ada_boost_optuna = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    score = cross_val_score(ada_boost_optuna, X_train_scaled, y_train, cv=5).mean()
    return score


study = optuna.create_study(direction='maximize', study_name='AdaBoost')
study.optimize(objective, n_trials=100)

adaboost_optuna_score = study.best_value
adaboost_optuna_params = study.best_params

scores.loc['AdaBoost', 'Score'] = adaboost_optuna_score
scores.loc['AdaBoost', 'Model_params'] = str([f'{k}: {v}' for k, v in adaboost_optuna_params.items()])

In [ ]:
adaboost_optuna_params

In [ ]:
adaboost = AdaBoostClassifier(n_estimators=400, learning_rate=1.14718573810009, random_state=42)
model_score(adaboost)

In [ ]:
adaboost.fit(X_train_scaled, y_train)
y_pred_adaboost = adaboost.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/titanic/test.csv')['PassengerId']
submission_adaboost = pd.DataFrame({'PassengerId': idx, 'Survived': y_pred_adaboost})
submission_adaboost.to_csv("submission_adaboost.csv", index = False)
print('✅ submission_adaboost.csv created successfully!')

# результат на test - 0.77272 с кодированием Pclass
# результат на test - 0.77511 без кодирования Pclass

**4.5 GradienBoosting**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

def objective(trial):
    loss = trial.suggest_categorical('loss', ['log_loss', 'exponential'])
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20, step=1)
    max_depth = trial.suggest_int('max_depth', 1, 15, step=1)
    grad_boost_optuna = GradientBoostingClassifier(loss=loss, learning_rate=learning_rate, 
                                                    n_estimators=n_estimators, min_samples_split=min_samples_split,
                                                    min_samples_leaf=min_samples_leaf, max_depth=max_depth, 
                                                    random_state=42)
    score = cross_val_score(grad_boost_optuna, X_train_scaled, y_train, cv=5).mean()
    return score


study = optuna.create_study(direction='maximize', study_name='Gradient Boosting')
study.optimize(objective, n_trials=100)

grad_boost_optuna_score = study.best_value
grad_boost_optuna_params = study.best_params

scores.loc['GradientBoosting', 'Score'] = grad_boost_optuna_score
scores.loc['GradientBoosting', 'Model_params'] = str([f'{k}: {v}' for k, v in grad_boost_optuna_params.items()])

In [ ]:
grad_boost_optuna_params

In [ ]:
grad_boost = GradientBoostingClassifier(loss='exponential', learning_rate=0.03166304188326599, n_estimators=150,
                                        min_samples_split=15,  min_samples_leaf=12, max_depth=8, random_state=42)
model_score(grad_boost)

In [ ]:
grad_boost.fit(X_train_scaled, y_train)
y_pred_grad_boost = grad_boost.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/titanic/test.csv')['PassengerId']
submission_grad_boost = pd.DataFrame({'PassengerId': idx, 'Survived': y_pred_grad_boost})
submission_grad_boost.to_csv("submission_grad_boost.csv", index = False)
print('✅ submission_grad_boost.csv created successfully!')

# результат на test - 0.77990 с кодированием Pclass
# результат на test - 0.76794 без кодирования Pclass

**4.6 CatBoost**

In [ ]:
from catboost import CatBoostClassifier

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=10)
    max_depth = trial.suggest_int('max_depth', 1, 15, step=1)
    cat_boost_optuna = CatBoostClassifier(learning_rate=learning_rate, n_estimators=n_estimators, 
                                          max_depth=max_depth, verbose=False, random_state=42)
    score = cross_val_score(cat_boost_optuna, X_train_scaled, y_train, cv=5).mean()
    return score


study = optuna.create_study(direction='maximize', study_name='CatBoost')
study.optimize(objective, n_trials=100)

cat_boost_optuna_score = study.best_value
cat_boost_optuna_params = study.best_params

scores.loc['CatBoost', 'Score'] = cat_boost_optuna_score
scores.loc['CatBoost', 'Model_params'] = str([f'{k}: {v}' for k, v in cat_boost_optuna_params.items()])

In [ ]:
cat_boost_optuna_params

In [ ]:
cat_boost = CatBoostClassifier(learning_rate=0.6280243661841942, n_estimators=110,
                                          max_depth=6, verbose=False, random_state=42)
model_score(cat_boost)

In [ ]:
cat_boost.fit(X_train_scaled, y_train)
y_pred_cat_boost = cat_boost.predict(X_test_scaled)
idx = pd.read_csv('/kaggle/input/titanic/test.csv')['PassengerId']
submission_cat_boost = pd.DataFrame({'PassengerId': idx, 'Survived': y_pred_cat_boost})
submission_cat_boost.to_csv("submission_cat_boost.csv", index = False)
print('✅ submission_cat_boost.csv created successfully!')

# результат на test - 0.76794 без кодирования Pclass